# Pre-processing di un Dataset di Rilevazione del Tumore al Seno

Si richiede di realizzare una pipeline di preprocessing volta a preparare un set di dati pulito e pronto per essere usato da modelli di ML.

Si richiede di realizzare 3 pipeline di trasformazioni diverse, che andranno poi inserite in un unico oggetto finale, che potrà poi essere esportato per essere utilizzato in produzione dai team di data scientist.

Le trasformazioni richieste sono:
1. **Pre-processing per i Record con Target = 1**
  - Pulizia dei Valori Mancanti: La pulizia sarà distinta tra variabili simmetriche e asimmetriche. Per le variabili asimmetriche si utilizzeranno tecniche di riempimento che tengano conto della distribuzione dei dati, mentre per quelle simmetriche si opterà per metodi di riempimento più standard.
  - Simmetrizzazione delle Variabili Asimmetriche: Per garantire una distribuzione più bilanciata dei dati, verranno corretti i valori delle variabili asimmetriche mediante tecniche di simmetrizzazione.
  - One-Hot Encoding delle Variabili Categoriche: Tutte le variabili categoriche saranno convertite in un formato numerico utilizzando il one-hot encoding, rendendo i dati utilizzabili nei modelli di machine learning.
  - Riscalatura mediante Standardizzazione: Le variabili numeriche saranno scalate usando la standardizzazione per garantire che tutte le variabili abbiano una distribuzione con media zero e deviazione standard pari a uno.
2. **Pre-processing per Tutti i Record del Dataset**
  - Pulizia dei Valori Mancanti: Sarà adottata una strategia personalizzata per riempire i valori mancanti in modo coerente con la natura delle variabili.
  - Discretizzazione a 20 Bin delle Variabili Numeriche: Le variabili numeriche verranno discretizzate in 20 bin per ridurre la complessità dei dati e facilitare l'analisi.
  - Encoding Ordinale delle Variabili Categoriche: La variabile categorica sarà codificata in base a un ordine crescente (A, B, C), mantenendo la semantica tra i valori.
  - Selezione delle 5 Variabili più Informative: Al termine delle trasformazioni, verranno selezionate le cinque variabili più informative rispetto al target, utilizzando una metrica appropriata, migliorando così l'efficienza e la precisione dei modelli successivi.
3. **Pre-processing delle Variabili Numeriche**
  - Pulizia dei Valori Mancanti: Come nella pipeline precedente, verrà scelto un metodo di pulizia adeguato alle variabili numeriche.
  - Principal Component Analysis (PCA): Verrà applicata una PCA per ridurre la dimensionalità del dataset, mantenendo l'80% della varianza spiegata, il che permetterà di ridurre il rumore e migliorare le prestazioni dei modelli.
  - Simmetrizzazione: Come nella pipeline 1, anche qui si procederà con la simmetrizzazione delle variabili asimmetriche per migliorare la distribuzione.
  - Riscalatura mediante Normalizzazione: Infine, le variabili numeriche saranno normalizzate tra 0 e 1, per uniformare la scala e facilitare il processo di apprendimento dei modelli.

In [1]:
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import PowerTransformer, OneHotEncoder, StandardScaler, OrdinalEncoder, KBinsDiscretizer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA

In [3]:
df = pd.read_csv("https://proai-datasets.s3.eu-west-3.amazonaws.com/sample_dataset.csv")

In [4]:
X = df.drop('target', axis=1)

In [5]:
y = df['target']

## Pipeline 1 - Record con Target = 1

Definizione dei record da considerare per questa pipeline.

In [6]:
#filtro il df per il valore del target richiesto, e rimozione della colonna del target
X1 = df[df['target']==1].drop('target', axis=1)
#separazione tra variabili numeriche e categoriche
numerical_features = X1.select_dtypes(exclude=["object","category","bool"])
categorical_features = X1.select_dtypes(include=["object","category","bool"])

### Analisi esplorativa: definizione del grado di asimmetria delle variabili

Distinzione tra le variabili simmetriche ed asimmetriche del dataset, al fine di personalizzare le tecniche di pulizia dei dati e operazioni di simmetrizzazione.

In [7]:
#calcolo della skewness per le variabili numeriche
skewness = numerical_features.skew()

In [8]:
#definizione delle soglie di skewness da letteratura per individuare variabili simmetriche, asimmetriche e fortemente asimmetriche
symmetrical_features_threshold = 0.5
asymmetrical_features_threshold = 1

Definite le soglie, si procede ad identificare le variabili simmetriche, debolmente asimmetriche e fortemente asimmetriche.

In [9]:
#individuazione delle variabili simmetriche
symmetrical_features = skewness[skewness.abs() < symmetrical_features_threshold].index
#individuazione delle variabili debolmente asimmetriche
asymmetrical_features = skewness[(skewness.abs() >= symmetrical_features_threshold) & (skewness.abs() < asymmetrical_features_threshold)].index
#individuazione delle variabili fortemente asimmetriche
strongly_asymmetrical_features = skewness[skewness.abs() >= asymmetrical_features_threshold].index

Per valutare come intervenire sulle variabili debolmente asimmetriche, si analizzano i valori di skewness di tali variabili.

In [ ]:
X1[asymmetrical_features].skew()

,0
mean smoothness,0.801788
mean symmetry,0.650033
worst texture,0.705988


Per trattare queste variabili si sceglie di modificare le threshold impostate, inserendo un'unico valore che vada a discriminare tra variabili simmetriche ed asimmetriche. Il nuovo valore impostato è pari a 0.7.

Da notare che la scelta di tale valore è puramente arbitraria e non ha un'effettiva valenza scientifica, al fine di un'utilizzo pratico si richiede uno studio approfondito per capire se effettivamente utilizzabile per i proprio scopi di analisi e modellazione.

In [ ]:
new_simmetry_threshold = 0.7
#definizione aggiornata delle variabili simmetriche
simmetrycal_features = skewness[skewness.abs() <= new_simmetry_threshold].index
#definizione aggiornata delle variabili asimmetriche
asimmetrycal_features = skewness[skewness.abs() > new_simmetry_threshold].index

### Definizione del preprocessing per le variabili numeriche

Si organizza la strutturazione delle operazioni di preprocessing per tipologia di variabili.

A valle di tale analisi, si decide di procedere con un processo di data cleaning che sfrutti la sostituzione dei missing con valore mediano, per le variabili asimmetriche.

Inoltre per le variabili asimmetriche è chiesta la realizzazione di un'operazione di simmetrizzazione.

Si predispone una pipeline apposita che contenga la sequenza di queste due operazioni.

In [ ]:
#definizione della sequenza di trasformazioni da realizzare per le variabili asimmetriche
asymmetrical_transformer = Pipeline(steps=[
    ('data-cleaning', SimpleImputer(strategy='median')),
    ('simmetrizzazione', PowerTransformer(method='yeo-johnson'))
])

Per le variabili simmetriche quello che si deve realizzare è un processo di data-cleaning basato sulla sostituzione dei missing con valore medio.

Si potrebbe procede quindi a costruire un oggetto di preprocessing che vada a trattare le variabili numeriche nel loro complesso. Prima di fare questo però si analizzano le operazioni da realizzare per le variabili categoriche.

### Definizione del preprocessing delle variabili categoriche

Per le variabili categoriche si realizza un processo di data-cleaning mediante la sostituzione dei missing con il valore più probabile.
Inoltre è stato richiesto di realizzare l'operazione di encoding delle variabili categoriche usando un'operazione di One-hot encoding.

Si costruisce l'ogetto per realizzare tali tipo di trasformazioni.

In [ ]:
#definzione della sequenza della sequeza di operazioni da realizzare per il preprocessing delle variabili categoriche
categorical_transformer = Pipeline(steps=[
    ('data-cleaning', SimpleImputer(strategy='most_frequent')),
    ('encoding', OneHotEncoder(categories= [categorical_features[col].unique().tolist() for col in categorical_features.columns]))
])

### Costruzione del preprocessor complessivo

Considerando le varie operazioni di preprocessing da svolgere sulle variabili simmetriche, asimmetriche e categoriche, si realizza l'oggetto che consenta di trattarle prima della simmetrizzazione di tutto il dataset.

In [ ]:
preprocessor = ColumnTransformer(transformers = [
    ('symmetrical', SimpleImputer(strategy='mean'), symmetrical_features),
    ('asymmetrical', asymmetrical_transformer, asymmetrical_features),
    ('categorical', categorical_transformer, categorical_features.columns)
])

### Definizione della standardizzazione

A valle delle operazioni di preprocessing impostate finora, è possibile applicare a tutto il dataset le operazioni di standardizzazione.
A questo proposito si può realizzare una pipeline finale che preveda la sequenza delle operazioni di preprocessing e di standardizzazione, applicabili a tutto il dataset.

In [ ]:
pipeline1 = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('standardizzazione', StandardScaler())
])

## Pipeline 2 - Tutte le variabili del dataset

Definizione del set di dati.

In [ ]:
X2 = df.drop('target', axis=1)

Per la costruzione di questa pipeline si adotta lo stesso approccio della pipeline precedente:
  - divisione tra variabili numeriche e categoriche;
  - tra le numeriche, distinguere tra variabili simmetriche e asimmetriche;
  - per ciascun tipo individuato, realizzare la sequenza di trasformazioni più appropriata e combinare il tutto in un unico oggetto finale

In [ ]:
#separazione tra variabili numeriche e categoriche
numerical_features = X2.select_dtypes(exclude=["object","category","bool"])
categorical_features = X2.select_dtypes(include=["object","category","bool"])

### Variabili categoriche

Per le variabili categoriche si realizza una pulizia dei missing considerando l'utilizzo del valore più probabile e un encoding ordinale.

In [ ]:
categorical_preprocessor = Pipeline(steps=[
    ('data-cleaning', SimpleImputer(strategy='most_frequent')),
    ('encoding', OrdinalEncoder(categories=[['A','B','C']]))
])

### Variabili numeriche

Per le variabili numeriche si fanno delle considerazioni analoghe a quanto fatto precedentemente per la distinzione tra variabili simmetriche ed asimmetriche, e poi per ciascun tipo si provvede a costruire degli opportuni oggetti che compongono l'insieme di trasformazioni da realizzare.

Si procede ad individuare le variabili simmetriche e quelle asimmetriche, utilizzando le stesse soglie definite precedentemente.

In [ ]:
#calcolo della skewness
skewness = numerical_features.skew()
#per semplicità si usa la stessa soglia definita precedentemente per fare da discriminante tra variabili simmetriche ed asimmetriche
#definizione aggiornata delle variabili simmetriche
simmetrycal_features = skewness[skewness.abs() <= new_simmetry_threshold].index
#definizione aggiornata delle variabili asimmetriche
asimmetrycal_features = skewness[skewness.abs() > new_simmetry_threshold].index

La discriminante tra i due tipi di variabili è servita al fine di adottare per le due tipologie diverse due tecniche diverse di data cleaning: usando la media per le simmetriche, usando il valore mediano per le asimmetriche.

Successivamente le variabili subiranno il processo di discretizzazione nel numero indicato di Bin.

In [ ]:
cleaning_transformer2 = ColumnTransformer(transformers=[
    ('symmetrical', SimpleImputer(strategy='mean'), simmetrycal_features),
    ('asymmetrical', SimpleImputer(strategy='median'), asimmetrycal_features)
])

In [ ]:
numerical_preprocessor = Pipeline(steps=[
    ('data-cleaning',cleaning_transformer2),
    ('discretizer', KBinsDiscretizer(n_bins=20, strategy='quantile', encode='ordinal'))
])

### Definizione della pipeline complessiva

Per completare la pipeline2 è necessario definire correttamente la sequenza di trasformazioni da realizzare ed il tipo di variabili su cui esse devono agire.

In [ ]:
#definizione del preprocessor che dovrà agire sulle variabili del dataset prima della feature selection
preprocessor2 = ColumnTransformer(transformers=[
    ('categorical', categorical_preprocessor, categorical_features.columns),
    ('numerical', numerical_preprocessor, numerical_features.columns)
])

In [ ]:
#definizione della pipeline
pipeline2 = Pipeline(steps=[
    ('preprocessing', preprocessor2),
    ('feature-selection', SelectKBest(f_regression, k=5))
])

## Pipeline 3 - Variabili numeriche


In [ ]:
#selezione delle variabili al quale applicare la pipeline
X3 = df.select_dtypes(exclude=["object","category","bool"]).drop('target', axis=1)

Si fa la distinzione tra simmetriche ed asimmetriche analogamente a quanto fatto precedentemente: quindi si può usare la stessa distinzione fatta prima.

In [ ]:
symmetrical_preprocessor = Pipeline(steps=[
    ('data-cleaning', SimpleImputer(strategy='mean')),
    ('normalizer', MinMaxScaler())
])

asymmetrical_preprocessor = Pipeline(steps=[
    ('data-cleaning', SimpleImputer(strategy='median')),
    ('simmetrizer', PowerTransformer(method='yeo-johnson')),
    ('normalizer', MinMaxScaler())
])

Si procede ad applicare la PCA su entrambi i tipi di variabili trattatI, completando così l'ultima pipeline.

In [ ]:
pipeline3 = Pipeline(steps=[
    ('preprocessing',ColumnTransformer(transformers=[
        ('asymmetrical', asymmetrical_preprocessor, asimmetrycal_features),
        ('symmetrical', symmetrical_preprocessor, simmetrycal_features)
    ])),
    ('PCA', PCA(n_components=0.8))
])

## Definizione oggetto finale

Si combinano le tre pipeline definite in un unico oggetto finale, pronto per essere esportato.

In [ ]:
pipeline = ColumnTransformer(transformers=[
    ('step_1', pipeline1, X1.columns),
    ('step2', pipeline2, X2.columns),
    ('step3', pipeline3, X3.columns)
])

In [ ]:
pipeline.fit_transform(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 3 are removed. Consider decreasing the number of bins.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307

array([[ 1.65517851e-15,  1.34955792e+00,  1.13236903e+00, ...,
        -6.56079759e-01, -7.53035159e-02,  1.19667570e-01],
       [ 2.02210886e+00,  1.79266920e+00,  2.21644684e+00, ...,
        -1.71772761e-01, -1.00914682e-02,  7.85402330e-02],
       [ 1.74878595e+00,  1.66543923e+00,  1.80616508e+00, ...,
        -6.99946886e-02, -1.55417253e-01,  2.29929119e-01],
       ...,
       [ 7.89049800e-01,  7.13408054e-01, -2.65451582e-15, ...,
         4.26671224e-01,  7.87303734e-02,  1.00538418e-01],
       [ 2.03142669e+00,  2.10855051e+00,  2.01297377e+00, ...,
        -3.06778241e-01, -1.24457432e-01, -7.90657232e-02],
       [-1.95660312e+00, -1.93560772e+00, -1.60284266e+00, ...,
        -7.38097993e-02, -3.13320595e-01,  1.95820066e-01]])

In [ ]:
#si esporta l'oggetto finale in un file joblib
from joblib import dump
dump(pipeline, 'pipeline.joblib')

['pipeline.joblib']